# Cifar 10 with Transfer Learning
I have implemented cifar 10 with ANN, and CNN earlier, and in this notebook i will be trying to implement it using transfer learning and see how the results vary from my previous attempts <br> <br>
You can check the previous notebooks from here - 
- [Cifar 10 with ANN](https://www.kaggle.com/kambojharyana/playing-around-with-cifar-and-ann)
- [Cifar 10 with CNN, using Data Augementation](https://www.kaggle.com/kambojharyana/cifar10-with-cnn)

In [1]:
# load the standart libraries
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)



In [2]:
#Import keras functions

from keras import Sequential

from keras.applications import VGG19,ResNet50

'Import the datagenerator to augment images'
from keras.preprocessing.image import ImageDataGenerator

from keras.optimizers import SGD,Adam
from keras.callbacks import ReduceLROnPlateau

from keras.layers import Flatten,Dense,BatchNormalization,Activation,Dropout

'Import to_categorical from the keras utils package to one hot encode the labels'
from keras.utils import to_categorical

In [3]:
#Import dataset
from keras.datasets import cifar10

# load the dataset into train and test from cifar10 object
(X_train,y_train),(X_test,y_test)=cifar10.load_data()

170500096/170498071 [==============================] - 2s 0us/step


In [4]:
# view the dimensions of train and test data to ensure everything is going fine 
print(X_train.shape,y_train.shape)
print(X_test.shape,y_test.shape)

(50000, 32, 32, 3) (50000, 1)
(10000, 32, 32, 3) (10000, 1)


The data visualization part has been done in the previos two notebooks, here, we will jump straight into model training, before done some preprocessing

In [5]:
num_classes = 10
y_train = to_categorical(y_train,num_classes)
y_test = to_categorical(y_test,num_classes)

In [6]:
# checking the dimensions again, to verify the change
print(X_train.shape,y_train.shape)
print(X_test.shape,y_test.shape)
# the dimensions of the labels have now changed. 

(50000, 32, 32, 3) (50000, 10)
(10000, 32, 32, 3) (10000, 10)


# Model 

The preprocessing part is done, now, before defining the transfer learning model, we will first define the data augmentation object, and train it on the training data only. We will be using data augmentation of training data only. 

In [7]:
train_generator = ImageDataGenerator(
                                    rotation_range=2, 
                                    horizontal_flip=True,
                                    zoom_range=.1 )


#now fit it to train data 
train_generator.fit(X_train)

## Building model

In [8]:
# will be using VGG19, make sure the input shape is same as that of CIFAR10
base_model = VGG19(include_top=False,weights='imagenet',input_shape=(32,32,3),classes=y_train.shape[1])

80142336/80134624 [==============================] - 1s 0us/step


In [9]:
# use the base model, and add more layers to it. 
model= Sequential()
model.add(base_model) #Adds the base model (in this case vgg19 to model)
model.add(Flatten()) 

# add some dense layers 
model.add(Dense(1024,activation=('relu')))
model.add(Dense(512,activation=('relu'))) 
model.add(Dense(256,activation=('relu'))) 
model.add(Dropout(.3)) #Adding a dropout layer that will randomly drop 30% of the weights
model.add(Dense(128,activation=('relu')))
model.add(Dropout(.2)) #Adding a dropout layer that will randomly drop 20% of the weights
model.add(Dense(10,activation=('softmax'))) #This is the classification layer


In [10]:
# model summay 
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
vgg19 (Functional)           (None, 1, 1, 512)         20024384  
_________________________________________________________________
flatten (Flatten)            (None, 512)               0         
_________________________________________________________________
dense (Dense)                (None, 1024)              525312    
_________________________________________________________________
dense_1 (Dense)              (None, 512)               524800    
_________________________________________________________________
dense_2 (Dense)              (None, 256)               131328    
_________________________________________________________________
dropout (Dropout)            (None, 256)               0         
_________________________________________________________________
dense_3 (Dense)              (None, 128)               3

In [11]:
# compile the model 
model.compile(optimizer='adam',loss='categorical_crossentropy',metrics=['accuracy'])

In [12]:
batch_size = 64
epochs = 50
history = model.fit_generator(train_generator.flow(X_train,y_train,batch_size=batch_size),
                      epochs=epochs,
                      steps_per_epoch=X_train.shape[0]//batch_size,
                      validation_data=(X_test,y_test))

Epoch 1/50
781/781 [==============================] - 44s 56ms/step - loss: 2.0582 - accuracy: 0.1915 - val_loss: 1.8485 - val_accuracy: 0.2409
Epoch 2/50
781/781 [==============================] - 44s 56ms/step - loss: 1.8229 - accuracy: 0.2520 - val_loss: 1.7105 - val_accuracy: 0.3252
Epoch 3/50
781/781 [==============================] - 44s 56ms/step - loss: 1.6597 - accuracy: 0.3452 - val_loss: 1.5429 - val_accuracy: 0.3793
Epoch 4/50
781/781 [==============================] - 44s 57ms/step - loss: 1.6066 - accuracy: 0.3716 - val_loss: 1.4968 - val_accuracy: 0.4077
Epoch 5/50
781/781 [==============================] - 44s 56ms/step - loss: 1.4737 - accuracy: 0.4223 - val_loss: 1.4448 - val_accuracy: 0.4321
Epoch 6/50
781/781 [==============================] - 43s 55ms/step - loss: 1.3379 - accuracy: 0.4836 - val_loss: 1.2239 - val_accuracy: 0.5319
Epoch 7/50
781/781 [==============================] - 44s 56ms/step - loss: 1.2888 - accuracy: 0.5257 - val_loss: 1.1839 - val_accuracy:

In [13]:
model.evaluate(X_test,y_test)

313/313 [==============================] - 3s 9ms/step - loss: 0.6757 - accuracy: 0.7978


[0.6757317185401917, 0.7978000044822693]

So, model with transfer learning is having around 80% of accuracy on test data, and around 84% of accuracy on train data with augmentation. 

Still, there are many things that we can try from here, like keeping the VGG layers fixed, adding Batch Normalization, data normalization, playing around with the final layers of the network and all. Lets cover that in the future. 